# Document Loading

In [ ]:
%pip install -q -U langchain langchain-community langchain-chroma langchain-openai faiss-cpu

# Set Up Environment

In [ ]:
# Load .env variables into the environment
import os
from pathlib import Path
from dotenv import load_dotenv  
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

## Load from a Directory

In [1]:
from langchain_community.document_loaders import DirectoryLoader 
from langchain_community.document_loaders.text import TextLoader 

loader = DirectoryLoader("docs/", glob="**/*.txt", loader_cls=TextLoader, show_progress=True) 
dirdocs = loader.load() 
len(dirdocs) 

100%|██████████| 2/2 [00:00<00:00, 1712.66it/s]


2

# Document Splitting

## Split Documents

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(dirdocs)

# Generate Embeddings

In [4]:
# Set up in-memory vector store
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
vector_store = FAISS.from_documents(chunks, OpenAIEmbeddings())



In [26]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [12]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain

@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vector_store.similarity_search_with_score(query, similarity_threshold=0.5))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [ ]:
result = retriever.invoke("What are the key things to know about the company?")
pretty_print_docs(result)

In [28]:
for doc in result:
    print(doc.metadata)


{'source': 'docs/darnell_summary.txt', 'score': 0.3832423}
Document 1:

6. Remind People of Their Why
   - Know and frequently remind team members of their motivations
   - Use their 'why' to keep them moving forward

7. Develop Independent Leaders
   - Aim to create leaders who can function without constant guidance
   - Success comes from helping others succeed

## Advice for New Associates
1. Set proper expectations (there will be more "no's" than "yes's" initially)
2. Utilize the established system and training
3. Focus on getting paid, promoted, and fast-start qualified
4. Share the opportunity through videos and scripts provided
5. Create a positive "first week story" to share with future recruits
6. Balance learning with earning
7. Strive for independence in running your business

## Keys to Success
- Consistent plugging into the system (attending events, trainings, etc.)
- Changing associations to those who empower and uplift you
- Having a vision that inspires others to follow

In [ ]:
query = "What are the key things to know about the company?"
matches = vector_store.similarity_search(query, k=5)
matches